In [1]:
!pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3 -q --no-cache
!pip install accelerate bitsandbytes
!pip install json-repair

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 28.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.

In [2]:
from huggingface_hub import login
login(new_session=False)

In [4]:
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from transformers import BitsAndBytesConfig
import re
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import json
from datetime import datetime
import torch
import shutil
from json_repair import repair_json as rj



model = Gemma3ForConditionalGeneration.from_pretrained(
    "google/gemma-3-4b-it",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="sdpa"
)
processor = AutoProcessor.from_pretrained(
    "google/gemma-3-4b-it",
    padding_side="left"
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [5]:
PromptListEng = ["Is the provided image showing a real face or a generated one?",  # 0
                 "Is the provided image showing a real face or a generated one? Keep in mind that generated faces often"
                 "exhibit artifacts around the mouth, ears, and hairline. Inconsistent lighting and shading may also be"
                 "present.",  # 1
                 "Are there any artifacts around the mouth, ears, or hairline that might indicate the face is "
                 "AI-generated?",  # 2
                 "Is the provided image showing a real face or a generated one? I will pay you $1,000 for the correct "
                 "answer.",  # 3
                 "This is a life-threatening situation. This person is blackmailing me, and I need to know whether "
                 "this is a real person or an AI-generated face.",  # 4
                 "I believe this person is real. Based on the image, does it appear to be a real face or a generated "
                 "one?",  # 5
                 "I believe this person doesn’t exist. Based on the image, does it appear to be a real face or a "
                 "generated one?"  # 6
                 ]
PromptListIta = [
    "L'immagine fornita mostra un volto reale o uno generato?",
    "L'immagine fornita mostra un volto reale o uno generato? Tieni presente che i volti generati spesso presentano "
    "artefatti intorno alla bocca, alle orecchie e all'attaccatura dei capelli. Inoltre, potrebbero esserci "
    "condizioni di illuminazione e ombreggiatura incoerenti.",
    "Ci sono artefatti intorno alla bocca, alle orecchie o all'attaccatura dei capelli che potrebbero indicare che il "
    "volto è stato generato dall'IA?",
    "L'immagine fornita mostra un volto reale o uno generato? Ti pagherò 1.000 dollari per una risposta corretta.",
    "Questa è una situazione che mette a rischio la mia vita. Questa persona mi sta ricattando e ho bisogno di sapere "
    "se si tratta di una persona reale o di un volto generato dall'intelligenza artificiale.",
    "Credo che questa persona sia reale. In base all'immagine, ti sembra un volto reale o uno generato?",
    "Credo che questa persona non esista. In base all'immagine, ti sembra un volto reale o uno generato?"
]


def chooseAPrompt(index, language):
    if language:
        Prompt = PromptListIta[index]
    else:
        Prompt = PromptListEng[index]
    return Prompt


def getSystemPrompt(isItalian, uncertainEn):
    if uncertainEn:
        if isItalian:
            return (
                "Sei un sistema professionale di classificazione delle immagini che analizza le immagini fornite per "
                "determinare se mostrano un volto umano reale o uno generato.\n"
                "Rispondi SOLO usando il seguente formato JSON:\n"
                "{\n"
                "  \"result\": \"[real face]\" oppure \"[generated]\"or [uncertain], \n"
                "  \"explanation\": \"Una spiegazione precisa basata su indizi visivi osservati nell'immagine.\"\n"
                "}\n"
                "NON includere frasi generiche come 'non ci sono artefatti'. Sii visivo e dettagliato nella tua analisi."
                "Rispondi ESCLUSIVAMENTE con questo JSON, senza commenti o testo extra."
            )
        else:
            return (
                "You are a professional image classification system that analyzes provided images to determine if they "
                "show a real human face or a generated one.\n"
                "Respond ONLY with the following JSON format:\n"
                "{\n"
                "  \"result\": \"[real face]\" or \"[generated]\" or \"[uncertain]\",\n"
                "  \"explanation\": \"A precise explanation based on observed visual clues from the image.\"\n"
                "}\n"
                "Do NOT include generic phrases like 'there are no artifacts'. Be visual and detailed in your analysis."
                "Reply ONLY with this JSON, no extra text."
            )
    else:
        if isItalian:
            return (
                "Sei un sistema professionale di classificazione delle immagini che analizza le immagini fornite per "
                "determinare se mostrano un volto umano reale o uno generato.\n"
                "Rispondi SOLO usando il seguente formato JSON:\n"
                "{\n"
                "  \"result\": \"[real face]\" oppure \"[generated]\" \n"
                "  \"explanation\": \"Una spiegazione precisa basata su indizi visivi osservati nell'immagine.\"\n"
                "}\n"
                "NON includere frasi generiche come 'non ci sono artefatti'. Sii visivo e dettagliato nella tua analisi."
                "Rispondi ESCLUSIVAMENTE con questo JSON, senza commenti o testo extra."
            )
        else:
            return (
                "You are a professional image classification system that analyzes provided images to determine if they "
                "show a real human face or a generated one.\n"
                "Respond ONLY with the following JSON format:\n"
                "{\n"
                "  \"result\": \"[real face]\" or \"[generated]\",\n"
                "  \"explanation\": \"A precise explanation based on observed visual clues from the image.\",\n"
                "}\n"
                "Do NOT include generic phrases like 'there are no artifacts'. Be visual and detailed in your analysis."
                "Reply ONLY with this JSON, no extra text."
            )


def usingOneShot():
    oneShotMessages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": "/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/test/fake/0A266M95TD.jpg"},
                {"type": "text", "text": "Is the provided image showing a real face or a generated one?"}
            ]
        },
        {
            "role": "assistant",
            "content": [
                {"type": "text", "text": '''
    {
      "result": "generated",
      "explanation": "The image shows distinctive characteristics of artificially generated faces, typical of GAN models (such as StyleGAN). First, there are slight asymmetries in the glasses: the right and left parts are not perfectly aligned, a common detail in synthetic faces. Second, the background appears blurred and lacks realistic depth, with no distinguishable elements, showing an unnatural blending of hair and head contours. Additionally, the lighting on the face is too uniform and lacks consistent shadows; for example, there are no shadows cast by the glasses, as would be expected in a real photo. The teeth are overly regular and symmetrical, and the eyes appear perfectly centered and devoid of complex reflections or imperfections, which are normally found in real human faces. All these subtle signals combined strongly indicate that this is an artificially generated face."
    }
                '''}
            ]
        }
    ]
    return oneShotMessages

def createOneShot(exampleImage, isFake, imagePath, prompt, isItalian, systemPrompt=None):
    """
    Crea un esempio one-shot multimodale compatibile con Gemma3 e AutoProcessor.
    """
    if isItalian:
        note_example = "Nota: Questa immagine è falsa (generata artificialmente)." if isFake else "Nota: Questa immagine è reale."
        note_task = "Sapendo questo, analizza la seconda immagine e rispondi alla seguente domanda:"
        example_answer = "Questa immagine è reale." if not isFake else "Questa immagine è falsa."
    else:
        note_example = "Note: This image is fake (AI-generated)." if isFake else "Note: This image is real."
        note_task = "Knowing this, analyze the second image and answer the following question:"
        example_answer = "This image is real." if not isFake else "This image is fake."

    messages = []

    # Esempio (immagine + nota)
    messages.append({
        "role": "user",
        "content": [
            {"type": "image", "image": str(exampleImage)},
            {"type": "text", "text": prompt}
        ]
    })

    # Risposta dell’esempio
    messages.append({
        "role": "assistant",
        "content": [
            {"type": "text", "text": example_answer}
        ]
    })

    # Target (immagine + prompt finale)
    messages.append({
        "role": "user",
        "content": [
            {"type": "image", "image": str(imagePath)},
            {"type": "text", "text": f"{note_task}\n{prompt}"}
        ]
    })

    return messages


In [6]:
def loadDataset(numImages):
    fake_dir = Path("/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/test/fake")
    real_dir = Path("/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/test/real")
    half = numImages // 2
    fake_images = list(fake_dir.glob("*.jpg"))[:half]
    real_images = list(real_dir.glob("*.jpg"))[:half]
    images_with_labels = [(img, 1) for img in real_images] + [(img, 0) for img in fake_images]
    return images_with_labels, len(fake_images), len(real_images)


def shuffleDataset(dataset):
    random.shuffle(dataset)


def saveDataset(dataset, name):
    base_dir = Path("/kaggle/working/my_dataset") / name
    real_dir = base_dir / "real"
    fake_dir = base_dir / "fake"

    # Crea le directory se non esistono
    real_dir.mkdir(parents=True, exist_ok=True)
    fake_dir.mkdir(parents=True, exist_ok=True)

    # Salva le immagini nelle rispettive cartelle
    for i, (img_path, label) in enumerate(dataset):
        if label == 1:
            dest = real_dir / f"real_{i}{img_path.suffix}"
        else:
            dest = fake_dir / f"fake_{i}{img_path.suffix}"

        shutil.copy(img_path, dest)


def loadExistingDataset(folder_name):
    # Costruisce il path assoluto alla cartella dentro /kaggle/input/
    base_dir = Path("/kaggle/input") / folder_name
    real_dir = base_dir / "real"
    fake_dir = base_dir / "fake"

    # Verifica che le directory esistano
    if not real_dir.exists() or not fake_dir.exists():
        raise FileNotFoundError(
            f"La cartella '/kaggle/input/{folder_name}' non è strutturata correttamente (manca 'real/' o 'fake/')")

    # Carica le immagini con etichette
    real_images = [(img_path, 1) for img_path in real_dir.glob("*.jpg")]
    fake_images = [(img_path, 0) for img_path in fake_dir.glob("*.jpg")]

    dataset = real_images + fake_images
    return dataset, len(fake_images), len(real_images)


In [7]:
import numpy as np
def analyze_images_batch(
    img_paths, labels, prompt, modelName,
    fewShot, few_shot_messages, systemPrompt,
    counters, showImages
):
    """
    Analizza un batch di immagini contemporaneamente.
    img_paths: lista di Path
    labels: lista di 0/1 (fake/real)
    """
    batch_entries = []
    messages_batch = []

    # Costruzione dei messaggi per tutte le immagini
    for img_path, lab in zip(img_paths, labels):
        messages = [
            {"role": "system", "content": [{"type": "text", "text": systemPrompt}]}
        ]
        if fewShot:
            messages.extend(few_shot_messages)

        messages.append({
            "role": "user",
            "content": [
                {"type": "image", "image": str(img_path)},
                {"type": "text", "text": prompt}
            ]
        })
        messages_batch.append(messages)
        batch_entries.append({
            "image_path": str(img_path),
            "ground_truth": "real" if lab == 1 else "fake",
            "prediction": None,
            "explanation": None,
            "error": None,
            "embedding_mean": None,   # nuovo campo
        })

    try:
        # Prepara tensori batch
        inputs = processor.apply_chat_template(
            messages_batch,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to(model.device, dtype=torch.float32)

        input_lens = [len(ids) for ids in inputs["input_ids"]]

        with torch.inference_mode():
            # Ottenere hidden states
            outputs = model(
                **inputs,
                output_hidden_states=True,
                return_dict=True
            )
            hidden_states = outputs.hidden_states[-1].detach().float().cpu()

            # Generazione batch
            generations = model.generate(
                **inputs,
                max_new_tokens=256,
                do_sample=False
            )

        # Decodifica e parsing per ogni immagine
        for i, entry in enumerate(batch_entries):
            gen = generations[i][input_lens[i]:]
            output_text = processor.decode(gen, skip_special_tokens=True)

            print(f"\nImage: {Path(entry['image_path']).name}")
            print("Raw Output:", output_text)

            # Salva embeddings per questa immagine
            emb = hidden_states[i].mean(dim=0)  # [hidden_dim]
            entry["embedding_mean"] = emb.numpy().astype(np.float16).tolist()

            text_clean = re.sub(
                r"^```(?:json)?\s*([\s\S]*?)\s*```$",
                r"\1",
                output_text.strip(),
                flags=re.MULTILINE
            )
            text_clean = repair_json(text_clean)

            try:
                parsed = json.loads(text_clean)
                result = parsed.get("result")
                explanation = parsed.get("explanation", None)

                if isinstance(result, list) and result:
                    result = result[0]

                prediction = str(result).strip().lower()
                entry["prediction"] = prediction
                entry["explanation"] = explanation
            except Exception as e:
                counters["er"] += 1
                entry["error"] = f"JSON parsing error: {e}"
                print(f" JSON Parsing error: {e}")
                continue

            if showImages:
                Image.open(entry["image_path"]).show()

            # Decisione e valutazione
            lab = 1 if entry["ground_truth"] == "real" else 0
            if lab == 1:  # Real
                if prediction in {"real face", "real", "[real face]", "agreed", "[real]", "[no]", "no"}:
                    counters["tn"] += 1
                    print(" TN (real correctly identified)")
                elif prediction in {"generated", "[generated]", "didn't agree", "generated face", "[generated face]", "yes","[yes]"}:
                    counters["fp"] += 1
                    print(" FP (real misclassified as fake)")
                else:
                    counters["rejection_real"] += 1
                    print(" Rejection on real image")
            else:  # Fake
                if prediction in {"generated", "[generated]", "didn't agree", "generated face", "[generated face]", "yes","[yes]"}:
                    counters["tp"] += 1
                    print(" TP (fake correctly identified)")
                elif prediction in {"real face", "real", "[real face]", "agreed", "[real]", "[no]", "no"}:
                    counters["fn"] += 1
                    print(" FN (fake misclassified as real)")
                else:
                    counters["rejection_fake"] += 1
                    print(" Rejection on fake image")

    except Exception as e:
        print(f"Runtime error on batch: {e}")
        counters["er"] += len(img_paths)
        for entry in batch_entries:
            entry["error"] = f"Runtime error: {e}"

    return batch_entries, counters


def repair_json(text):
    # Bilancia parentesi graffe
    open_braces = text.count('{')
    close_braces = text.count('}')
    missing = open_braces - close_braces
    if missing > 0:
        text += '}' * missing

    # Tenta di chiudere virgolette aperte
    quote_count = text.count('"')
    if quote_count % 2 != 0:
        text += '"'

    # Rimuovi caratteri dopo ultima graffa chiusa (potrebbe esserci garbage)
    last_close = text.rfind('}')
    if last_close != -1:
        text = text[:last_close + 1]

    return text

In [8]:
def initMetrics():
    counters = {
        "tp": 0, "tn": 0, "fp": 0, "fn": 0, "er": 0,
        "rejection_real": 0, "rejection_fake": 0
    }
    return counters


def analizeMetrics(counters, images_with_labels, prompt, systemPrompt, oneShot, oneShotMessage, real_images,
                   fake_images,  imageBoost):
    total_classified = counters["tp"] + counters["tn"] + counters["fp"] + counters["fn"]
    accuracy = (counters["tp"] + counters["tn"]) / total_classified if total_classified else 0
    precision = counters["tp"] / (counters["tp"] + counters["fp"]) if (counters["tp"] + counters["fp"]) else 0
    recall = counters["tp"] / (counters["tp"] + counters["fn"]) if (counters["tp"] + counters["fn"]) else 0
    total_real = counters["tp"] + counters["fn"] + counters["rejection_real"]
    total_fake = counters["tn"] + counters["fp"] + counters["rejection_fake"]

    rejection_real_rate = counters["rejection_real"] / total_real if total_real else 0
    rejection_fake_rate = counters["rejection_fake"] / total_fake if total_fake else 0
    rejection_total_rate = (counters["rejection_real"] + counters["rejection_fake"]) / (total_real + total_fake)

    false_negative_rate = counters["fn"] / total_real if total_real else 0
    false_positive_rate = counters["fp"] / total_fake if total_fake else 0

    print("\n====== FINAL REPORT ======")
    print(f"Total processed: {len(images_with_labels)}")
    print(f"TP: {counters['tp']} | TN: {counters['tn']} | FP: {counters['fp']} | FN: {counters['fn']}")
    print(f"Rejections on real: {counters['rejection_real']} | Rejections on fake: {counters['rejection_fake']}")
    print(f"Text parsing errors: {counters['er']} ({(counters['er'] / len(images_with_labels)) * 100:.2f}%)\n")

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"False Negative Rate (real->fake): {false_negative_rate * 100:.2f}%")
    print(f"False Positive Rate (fake->real): {false_positive_rate * 100:.2f}%")
    print(f"Rejection Rate on real images: {rejection_real_rate * 100:.2f}%")
    print(f"Rejection Rate on fake images: {rejection_fake_rate * 100:.2f}%")

    results = {
        "total_processed": len(images_with_labels),
        "total_real": real_images,
        "total_fake": fake_images,
        "TP": counters["tp"],
        "TN": counters["tn"],
        "FP": counters["fp"],
        "FN": counters["fn"],
        "rejection_real": counters["rejection_real"],
        "rejection_fake": counters["rejection_fake"],
        "text_parsing_errors": counters["er"],
        "text_parsing_error_rate": (counters["er"] / len(images_with_labels)) if len(images_with_labels) else 0,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "false_negative_rate": false_negative_rate,
        "false_positive_rate": false_positive_rate,
        "rejection_real_rate": rejection_real_rate,
        "rejection_fake_rate": rejection_fake_rate,
        "rejection_total_rate": rejection_total_rate,
        "system_prompt": systemPrompt,
        "user_prompt": prompt,
        "oneShot": oneShot,
        "imageBoost": imageBoost
    }
    if oneShot:
        results["oneShotMessage"] = oneShotMessage
    return results


def saveAllJson(metrics, responses, PromptITA, modelName, i):
    outputData = {
        "metrics": metrics,
        "responses": responses
    }
    Path("resultsJSON").mkdir(exist_ok=True)

    # Imposta lingua
    language_tag = "ITA" if PromptITA else "ENG"

    # Timestamp per identificare diversi tentativi
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

    # Pulisci MODEL_NAME da caratteri non ammessi nei nomi file
    safe_model_name = modelName.replace(":", "_").replace("/", "_")

    # Costruisci filename
    filename = f"resultsJSON/real-vs-fake_{safe_model_name}_PromptType-{i}_{language_tag}_{timestamp}_result.json"

    # Salva JSON
    with open(filename, "w") as f:
        json.dump(outputData, f, indent=4)

    print(f"Results saved to {filename}.")

In [9]:
# VALORI PER CARICARE IL DATASET
saveDatasetDirectory = False
startMiniDt = False
NAME = "oneshot/test_OS"
MAX_IMAGES = 300
SHUFFLE = False
# ======================================

# VALORI PER IL PROMPT
INDEX_PROMPT = 4  # (0-6)
IS_ITALIAN = False
SHOW_IMAGES = False
ONESHOT = False
UNCERTAIN_EN = True
IMAGE_BOOST= False
# ===================================
# VALORI PER IL MODELLO

MODEL_NAME = "gemma3:4b"
# ===================================
# MODALITA' AUTOMATICA
AUTO_ON = False

# MODALITA' ONESHOT
ONESHOT = False
TAG_IMAGE = True
ONE_SHOT_IMAGE="/kaggle/input/oneshot/photoEx/real/man.jpg"

In [ ]:
if startMiniDt:
    images_with_labels, fakes, reals = loadDataset(MAX_IMAGES)
    saveDataset(images_with_labels, NAME)
else:
    images_with_labels, fakes, reals = loadExistingDataset(NAME)
    

if SHUFFLE:
    shuffleDataset(images_with_labels)

# prompt section
oneShotMessage = None
if TAG_IMAGE:
    isFake = False
else:
    isFake = True
if AUTO_ON:
    for INDEX_PROMPT in range(7):  # Prompt da 0 a 6
        for IS_ITALIAN in [False, True]:  # Prima inglese, poi italiano
            userPrompt = chooseAPrompt(INDEX_PROMPT, IS_ITALIAN)
            systemPrompt = getSystemPrompt(IS_ITALIAN, UNCERTAIN_EN)

            print(f"\n--- Prompt {INDEX_PROMPT} | Language: {'Italian' if IS_ITALIAN else 'English'} ---")
            print("Prompt:", userPrompt)

            # Inizializza le metriche
            counters = initMetrics()
            results = []

            # Configura il batch size
            BATCH_SIZE = 4  # puoi adattarlo in base alla memoria disponibile

            # Ciclo principale di analisi immagini con batching
            for i in tqdm(range(0, len(images_with_labels), BATCH_SIZE),
                          desc=f"Analyzing images in batches (Prompt {INDEX_PROMPT}, {'IT' if IS_ITALIAN else 'EN'})"):
                batch = images_with_labels[i:i+BATCH_SIZE]
                batch_paths = [x[0] for x in batch]
                batch_labels = [x[1] for x in batch]

                batch_results, counters = analyze_images_batch(
                    batch_paths,
                    batch_labels,
                    userPrompt,
                    MODEL_NAME,
                    ONESHOT,
                    oneShotMessage,
                    systemPrompt,
                    counters,
                    SHOW_IMAGES
                )

                results.extend(batch_results)

            # Salvataggio dei risultati
            jsonINFO = analizeMetrics(
                counters, images_with_labels, userPrompt,
                systemPrompt, ONESHOT, oneShotMessage,
                reals, fakes, IMAGE_BOOST
            )

            saveAllJson(jsonINFO, results, IS_ITALIAN, MODEL_NAME, INDEX_PROMPT)

else:
    userPrompt = chooseAPrompt(INDEX_PROMPT, IS_ITALIAN)
    systemPrompt = getSystemPrompt(IS_ITALIAN, UNCERTAIN_EN)
    
    print("you choose " + userPrompt)
    # initialize metrics
    counters = initMetrics()
    results = []
    # Configura il batch size
    BATCH_SIZE = 2  # puoi adattarlo in base alla memoria disponibile
    
    # Main analysis loop a batch
    for i in tqdm(range(0, len(images_with_labels), BATCH_SIZE), desc="Analyzing images in batches"):
        batch = images_with_labels[i:i+BATCH_SIZE]
        batch_paths = [x[0] for x in batch]
        batch_labels = [x[1] for x in batch]
    
        batch_results, counters = analyze_images_batch(
            batch_paths, 
            batch_labels, 
            userPrompt, 
            MODEL_NAME, 
            ONESHOT, 
            oneShotMessage,
            systemPrompt, 
            counters, 
            SHOW_IMAGES
        )
    
        results.extend(batch_results)
    jsonINFO = analizeMetrics(counters, images_with_labels, userPrompt, systemPrompt, ONESHOT, oneShotMessage,
                                  reals, fakes, IMAGE_BOOST)
    saveAllJson(jsonINFO, results, IS_ITALIAN, MODEL_NAME, INDEX_PROMPT)

you choose This is a life-threatening situation. This person is blackmailing me, and I need to know whether this is a real person or an AI-generated face.


Analyzing images in batches:   0%|          | 0/150 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:651: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Analyzing images in batches:   1%|          | 1/150 [00:38<1:35:18, 38.38s/it]


Image: real_111.jpg
Raw Output: ```json
{
  "result": "[real face]",
  "explanation": "The image exhibits natural skin texture with visible pores and wrinkles around the eyes and mouth, indicating a real human face. The lighting and shadows are consistent with a photograph, and the details of the eyes, teeth, and hair are realistic. There are subtle asymmetries in the face, such as the slight difference in the eyebrows and the shape of the nose, which are characteristic of human faces."
}
```
 TN (real correctly identified)

Image: real_59.jpg
Raw Output: ```json
{
  "result": "real face",
  "explanation": "The image exhibits subtle imperfections characteristic of a real human face, including asymmetrical features (slightly uneven eyebrows and a subtle asymmetry in the mouth), natural skin texture with visible pores and fine wrinkles around the eyes and mouth, and realistic shading and highlights. The details of the clothing and background are also consistent with a photograph of a re

Analyzing images in batches:   1%|▏         | 2/150 [01:24<1:46:26, 43.15s/it]


Image: real_45.jpg
Raw Output: ```json
{
  "result": "real face",
  "explanation": "The image exhibits natural skin texture with visible pores and fine lines around the eyes and mouth, indicating a real human face. The lighting is consistent, and the details of the eyes, nose, and mouth are well-defined, displaying realistic reflections and subtle variations in color. The smile appears genuine, and the overall composition suggests a photograph taken in a natural setting."
}
```
 TN (real correctly identified)

Image: real_4.jpg
Raw Output: ```json
{
  "result": "real face",
  "explanation": "The image exhibits several characteristics indicative of a real human face. The skin texture displays natural variations in tone and subtle imperfections, such as pores and fine lines around the eyes and mouth. The lighting creates realistic shadows under the eyebrows and along the cheekbones. The eyes show a natural glint and reflection, and the smile reveals slight asymmetry in the lips, consist

...


Image: fake_247.jpg
Raw Output: ```json
{
  "result": "[real face]",
  "explanation": "The image exhibits subtle imperfections consistent with a real photograph of a human face. There's a slight asymmetry in the eyebrows and eyelids, a natural variation in skin texture with visible pores and fine lines around the eyes and mouth, and a realistic reflection in the eyes. The lighting and shadows are uneven, suggesting a captured moment rather than a perfectly rendered image. The overall detail and complexity of the facial features are indicative of a genuine human subject."
}
```
 FN (fake misclassified as real)

Image: fake_156.jpg
Raw Output: ```json
{
  "result": "real face",
  "explanation": "The image exhibits natural skin texture with visible pores and fine lines around the eyes and mouth, indicative of a real human face. The lighting and shadows are consistent and realistic, and the subject's expression – a genuine smile – appears authentic. There's a subtle redness in the cheeks,

Analyzing images in batches:  81%|████████  | 121/150 [1:23:04<19:30, 40.36s/it]


Image: fake_225.jpg
Raw Output: ```json
{
  "result": "real face",
  "explanation": "The image exhibits realistic skin texture with visible pores and wrinkles around the eyes and mouth. The lighting and shadows are consistent with natural illumination, creating subtle variations in tone. The eyes have a natural depth and reflection, and the teeth show slight imperfections. The overall facial structure and proportions appear anatomically correct, suggesting a genuine human subject."
}
```
 FN (fake misclassified as real)

Image: fake_275.jpg
Raw Output: ```json
{
  "result": "real face",
  "explanation": "The image exhibits natural skin texture with visible pores and subtle variations in color, indicating a real human face. The lighting is consistent across the face, and the eyes show realistic reflections. The smile lines around the mouth and the slight asymmetry in the face are also characteristic of a genuine human expression."
}
```
 FN (fake misclassified as real)


Analyzing images in batches:  81%|████████▏ | 122/150 [1:23:42<18:32, 39.73s/it]


Image: fake_215.jpg
Raw Output: ```json
{
  "result": "real face",
  "explanation": "The image exhibits natural skin texture with visible pores and subtle variations in color, indicating a real human face. The lighting and shadows are consistent with a photograph, and the details of the eyes, nose, and mouth are realistically rendered. The hair has a natural, slightly messy appearance, and the facial expression shows subtle muscle movements, further supporting its authenticity."
}
```
 FN (fake misclassified as real)

Image: fake_155.jpg
Raw Output: ```json
{
  "result": "real face",
  "explanation": "The image exhibits natural skin texture with visible pores and fine lines around the eyes and mouth, indicating a real human face. The lighting and shadows are consistent with a photograph, and the details of the eyes, nose, and mouth are realistically rendered. There are subtle variations in color and tone that are characteristic of a biological subject."
}
```
 FN (fake misclassified a